<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/Gemini_LangChain/Take_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U --quiet langchain-google-genai  langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install --quiet langchain-google-genai

In [ ]:
from langchain import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document


from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [ ]:
import google.generativeai as genai

# Used to securely store your API key

genai.configure(api_key=userdata.get('Gemini_API_Key'))

model = genai.GenerativeModel('gemini-pro')

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# setup the gemini pro
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3, google_api_key=userdata.get('Gemini_API_Key'))

In [ ]:

result = gemini_llm.invoke("what is google vertex ai?")
print(result.content)


Google Vertex AI is a managed machine learning platform that provides a comprehensive set of tools and services to help businesses build, deploy, and manage machine learning models. It offers a wide range of features, including:

* **Model training:** Vertex AI provides a variety of tools and services to help businesses train machine learning models, including:
    * **AutoML:** AutoML is a set of tools that allow businesses to train machine learning models without having to write any code.
    * **Custom training:** Businesses can also use Vertex AI to train custom machine learning models using their own data and algorithms.
* **Model deployment:** Vertex AI provides a variety of options for deploying machine learning models, including:
    * **Model serving:** Businesses can deploy machine learning models as web services, which can be accessed by other applications.
    * **Batch prediction:** Businesses can also use Vertex AI to perform batch predictions on large datasets.
* **Model

## Example with a Prompt Template

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

**Gemini doesn’t support SystemMessage at the moment, but it can be added to the first human message in the row.**

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "return the sentiment for the given text in json fromat the sentiment value can be 'nagative','positive'"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

In [ ]:
chat_message =  chat_template.format_messages(text="Weather today is sunny and beautiful")

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3, convert_system_message_to_human=True,google_api_key=userdata.get('Gemini_API_Key') ) # set the convert_system_message_to_human to true


In [ ]:

llm.invoke(chat_message)

AIMessage(content='```json\n{\n  "sentiment": "positive"\n}\n```')

# Use Output parser to get formated output

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
chain = llm | parser

chat_message

[SystemMessage(content="return the sentiment for the given text in json fromat the sentiment value can be 'nagative','positive'"),
 HumanMessage(content='Weather today is sunny and beautiful')]

In [ ]:
sentiment = chain.invoke(chat_message)
print(sentiment)

{'sentiment': 'positive'}


# Tagging Chain with Gemini Pro

In [ ]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [ ]:
class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [ ]:

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Tags)

In [ ]:

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:

chain = prompt | llm | parser

chain.invoke({"query": "i don't like weather today."})

Tags(sentiment='sad', aggressiveness=2, language='english')

# Create a Conversational chain with Gemini pro

In [ ]:

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [ ]:

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hello! How may I assist you today?'

In [ ]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How may I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


"That's great to hear! I'm always happy to chat. What would you like to talk about?"

In [ ]:
PROMPT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI Assistant:
"""

PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=PROMPT_TEMPLATE
)

In [ ]:

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=PROMPT,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant")
)

In [34]:

conversation.predict(input="what is dojo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who is Gojo?
AI Assistant: Gojo Satoru is a character in the popular anime and manga series Jujutsu Kaisen. He is a powerful sorcerer and teacher at Jujutsu High, a school that trains students to combat supernatural beings known as Curses.

Gojo is known for his immense strength, unique abilities, and eccentric personality. He is also a mentor to the series' protagonist, Yuji Itadori.

Here are some specific details about Gojo:

* He is the strongest sorcerer in the world.
* He has a technique called "Limitless," which allows him to manipulate space and time.
* He is known for his distinctive white hair and blue eyes.
* He is a skilled swordsman and hand-to-hand

'A dojo is a training hall or gymnasium for martial arts. It is a place where people go to learn and practice martial arts, such as karate, judo, aikido, and taekwondo.\n\nDojos typically have a large open space for training, as well as smaller rooms for private lessons or specialized training. They may also have changing rooms, showers, and other amenities.\n\nDojos are typically run by a martial arts instructor, or sensei, who teaches students the techniques and principles of the martial art. Students typically train in the dojo several times a week, and may also participate in competitions or tournaments.\n\nHere are some specific details about dojos:\n\n* They are typically decorated with traditional Japanese symbols and artwork.\n* They may have a shrine or altar dedicated to the founder of the martial art.\n* Students typically wear a uniform, or gi, while training in the dojo.\n* Training in a dojo is typically very disciplined and structured.\n* Dojos are often seen as a place 

In [35]:
conversation.predict(input="What is dojo javascript library?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who is Gojo?
AI Assistant: Gojo Satoru is a character in the popular anime and manga series Jujutsu Kaisen. He is a powerful sorcerer and teacher at Jujutsu High, a school that trains students to combat supernatural beings known as Curses.

Gojo is known for his immense strength, unique abilities, and eccentric personality. He is also a mentor to the series' protagonist, Yuji Itadori.

Here are some specific details about Gojo:

* He is the strongest sorcerer in the world.
* He has a technique called "Limitless," which allows him to manipulate space and time.
* He is known for his distinctive white hair and blue eyes.
* He is a skilled swordsman and hand-to-hand

'Dojo is a free and open-source JavaScript library that provides a toolkit for building dynamic and interactive web applications. It is a modular library, meaning that you can include only the modules that you need for your project.\n\nDojo is known for its comprehensive set of features, including:\n\n* A powerful widget system for creating reusable UI components\n* A data store system for managing and manipulating data\n* A templating system for generating HTML markup\n* A routing system for managing application navigation\n* A REST API for interacting with web services\n\nDojo is also known for its strong community support and extensive documentation.\n\nHere are some specific details about Dojo:\n\n* It is written in JavaScript and is compatible with all major browsers.\n* It is used by a wide range of organizations, including Fortune 500 companies and government agencies.\n* It is licensed under the New BSD License, which is a permissive free software license.\n\nDojo is a powerful